In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import files
from sklearn.model_selection import cross_val_score
import time
import datetime
from sklearn.model_selection import KFold
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Проект 'рекомендательная система для магазина'/data.csv")

Invoices = df['Invoice'].unique()
indexes = []
for i in range(len(Invoices)):
    if Invoices[i][0] == 'C':
        indexes.append(i)
Invoices_n = np.delete(Invoices, indexes)
df = df.loc[df['Invoice'].isin(Invoices_n)]
df_c = df[['StockCode', 'Customer ID']].copy()
df_c = df_c.dropna()
df_c = df_c.reset_index(drop = True)

dicti = {}
for i in df_c['Customer ID'].unique():
    dicti[i] = (df_c[df_c['Customer ID'] == i]['StockCode']).tolist()

In [ ]:
one_buyers=[]
last_thing=[]
for i in dicti.keys():
    if len(dicti[i])==1:
        one_buyers.append(i)
for i in one_buyers:
    del dicti[i]
for i in dicti.keys():
    last_thing.append(dicti[i][-1])

data=[]
for i in dicti.keys():
    data.append(dicti[i])
len(data)

5767

In [ ]:
from sklearn.model_selection import train_test_split
from gensim.models import word2vec
import pickle
from google.colab import files

model = word2vec.Word2Vec(data, size=150, window=3, workers=4)

#создадим словарь со словами и соответсвующими им векторами
w2v = dict(zip(model.wv.index2word, model.wv.vectors))

In [ ]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

data_mean=mean_vectorizer(w2v).fit(data).transform(data)
data_mean.shape

(5767, 150)

In [ ]:
!pip install catboost 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_mean, last_thing, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
start_time = datetime.datetime.now()
fold = KFold(n_splits=5, shuffle=True, random_state=241)
parameters = {'n_estimators': [100]}
clf = RandomForestClassifier(random_state=241,n_jobs=-1)
print(clf.get_params().keys())
grid = GridSearchCV(clf, param_grid=parameters, scoring='accuracy', cv=fold)
grid.fit(x_train, y_train)
print('gs.best_score_:', grid.best_score_, '\n', 'gs.best_parameters_:', grid.best_params_)
print('Time elapsed:', datetime.datetime.now() - start_time)

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
import time
import datetime
from sklearn.model_selection import KFold
start_time = datetime.datetime.now()
params = {
      #"objective":"multiclass", 
      #"metric":"rmse",
      "learning_rate":0.005,
      "num_leaves":50,
      "min_child_samples":20,
      "sub_feature":0.4,
     "sub_row":1,
     "subsample_freq":0,
      "lambda_l1":4.6,
      "lambda_l2":1,
      "random_state":17,
       "n_estimators":100}
model = LGBMClassifier(**params,)
fold = KFold(n_splits=3, shuffle=True, random_state=241)
scores= cross_val_score(model, x_train, y_train, cv=fold, scoring='accuracy')
print(scores.mean())
print('Time elapsed:', datetime.datetime.now() - start_time)
#model.fit(x_train,y_train, verbose=True)

0.04920965554467902
Time elapsed: 0:04:28.495120


In [ ]:
#model.fit(x_train,y_train)
proba = model.predict_proba(x_test)

pos = 0
j=0
for i in proba:
  index = np.argpartition(i, -5)[-5:]
  classes = model.classes_[index]
  if y_test[j] in classes:
    pos += 1
  j += 1
accuracy = pos/len(y_test)

print("accuracy:",accuracy)

accuracy: 0.13258232235701906


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn import metrics
import time
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(data_mean, last_thing, test_size=0.33, random_state=42)
for C in [0.1,1,10]:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=C, random_state=241,multi_class='ovr', tol=0.001, n_jobs=-1)
    clf.fit(X_train,y_train)

    proba = clf.predict_proba(X_test)
    pos = 0
    j=0
    for i in proba:
      index = np.argpartition(i, -5)[-5:]
      classes = clf.classes_[index]
      if y_test[j] in classes:
        pos += 1
      j += 1
    accuracy = pos/len(y_test)
    print('Time elapsed:', datetime.datetime.now() - start_time)
    print("accuracy:",accuracy)    

Time elapsed: 0:04:41.813805
accuracy: 0.12920168067226892
Time elapsed: 0:05:58.040455
accuracy: 0.14390756302521007
Time elapsed: 0:10:46.783351
accuracy: 0.14653361344537816


In [ ]:
ml_file = open("clf.pkl",'wb')
pickle.dump(clf, ml_file)
files.download("clf.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn import metrics
import time
import datetime
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(data_mean, last_thing, test_size=0.33, random_state=42)

start_time = datetime.datetime.now()
fold = KFold(n_splits=5, shuffle=True, random_state=241)
parameters = {'C': [1]}
clf = LogisticRegression(random_state=241,multi_class='ovr', tol=0.001)
print(clf.get_params().keys())
print(sorted(metrics.SCORERS.keys()))
grid = GridSearchCV(clf, param_grid=parameters, scoring='accuracy', cv=fold)
grid.fit(X_train, y_train)
print('gs.best_score_:', grid.best_score_, '\n', 'gs.best_parameters_:', grid.best_params_)
print('Time elapsed:', datetime.datetime.now() - start_time)

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])
['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'r

In [ ]:
proba = grid.predict_proba(X_test)
start_time = datetime.datetime.now()
pos = 0
j=0
for i in proba:
  index = np.argpartition(i, -5)[-5:]
  classes = grid.classes_[index]
  if y_test[j] in classes:
    pos += 1
  j += 1
accuracy = pos/len(y_test)
print('Time elapsed:', datetime.datetime.now() - start_time)
print("accuracy:",accuracy)

Time elapsed: 0:00:00.060154
accuracy: 0.14758403361344538


In [ ]:
import json
import pickle
data_file = open("data.json",'w')
emb_file = open("word2vec.pkl",'wb')
ml_file = open("clf.pkl",'wb')
json.dump(dicti, data_file)
pickle.dump(model, emb_file)
pickle.dump(clf, ml_file)
files.download("data.json")
files.download("word2vec.pkl")
files.download("clf.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>